## Falcon-7b Text generation pipeline

This notebook provides [falcon-7b](https://huggingface.co/tiiuae/falcon-7b-instruct) playground for text generation.



In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

#### Modify the below cell to change the GPU and/or model to be used

In [ ]:
# Provide the id of the free GPU that you are intented to use
# Use "auto" instead of specifying integer id for gpu_id if unsure
# gpu_id = "auto"
gpu_id = 0

model = "tiiuae/falcon-7b-instruct"  # "tiiuae/falcon-7b" is also available

In [ ]:
# Use pre-trained tockeniser for the model
tokenizer = AutoTokenizer.from_pretrained(model)

# Use huggingface pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map=gpu_id,
)

In [ ]:
def get_response(text: str) -> list[dict]:
    """Get response from the hugging face pipeline.

    Args:
        text (str): Input text or context (prompt) to the model
    Returns:
        list[dict]: Output response from the model based on teh pipeline using.
                    Here, the key is 'generated_text' and the value is actual text generated.
    """
    sequences = pipeline(
        text,
        max_length=200,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )
    return sequences

In [ ]:
def print_response(text: str) -> None:
    """Parse and pretty print the response generated.

    Args:
        text (str): Input text or context (prompt) to the model
    Returns None
    """
    response = get_response(text)
    print(f"Prompt:\n\n{text}")
    print("---" * 30)
    print(f"Response:\n{response[0]['generated_text'][len(text):]}")

### Experiment by modifying the prompt in `sample_text*` and check the response

Note that the current pipeline is not suitable for conversation as it doesn't have the capability to remmeber the previous queries in the conversation.

In [ ]:
sample_text1 = """Three birds on a branch, if you use a shotgun and you shoot one of the birds,
how many birds there will be left on the branch?"""
print_response(sample_text1)

In [ ]:
sample_text2 = """We are talking about a loud shotgun."""
print_response(sample_text2)